In [ ]:
import torch

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import pandas as pd
import re

# Initialize an empty list to store data for the DataFrame
data = []

# Define the path to your text file
file_path = '/content/drive/MyDrive/Abreviation/CASI/CASI.txt'

# Read the file line by line
with open(file_path, 'r',encoding='utf-8', errors='ignore') as file:
  for line in file:
    # Split the line by the '|' delimiter
    parts = line.split('|')

    # Extract elements based on the expected format
    if len(parts) >= 7:  # Ensure the line has the expected number of fields
        abr = parts[0]
        long_form = parts[1]
        abr_text = parts[2]
        start_abr = int(parts[3])
        end_abr = int(parts[4])
        section = parts[5]  # May be empty if no section is specified
        text = parts[6]

        # Append the extracted fields to the data list as a dictionary
        data.append({
            'abr': abr,
            'long': long_form,
            'abr_text': abr_text,
            'start_abr': start_abr,
            'end_abr': end_abr,
            'section': section,
            'text': text
        })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Display the resulting DataFrame
print(df)


       abr                                      long abr_text  start_abr  \
0       AB                                  abortion      AB.        231   
1       AB                                  abortion      AB.        249   
2       AB                                  abortion       AB        223   
3       AB                                  abortion      AB.        194   
4       AB                                  abortion       AB        114   
...    ...                                       ...      ...        ...   
37495  VAD  vincristine adriamycin and dexamethasone     VAD.        139   
37496  VAD  vincristine adriamycin and dexamethasone     VAD,        172   
37497  VAD  vincristine adriamycin and dexamethasone      VAD        250   
37498  VAD  vincristine adriamycin and dexamethasone      VAD        181   
37499  VAD  vincristine adriamycin and dexamethasone      VAD         60   

       end_abr                         section  \
0          233                       

In [ ]:
dff=pd.DataFrame(df, columns=('abr','long','text'))

In [ ]:
# Calculate the mean length of the 'TEXT' column (number of characters)
max_text_length = dff['text'].apply(len).max()

# Calculate the mean length of the 'LABEL' column (number of characters)
max_label_length =dff['long'].apply(len).max()
# Calculate the mean length of the 'abr' column (number of characters)
max_abr_length =dff['abr'].apply(len).max()
# Print the results
print(f"Max length of 'TEXT': {max_text_length}")
print(f"Maxn length of 'LABEL': {max_label_length}")
print(f"Maxn length of 'LABEL': {max_abr_length}")
from transformers import BartTokenizer





Max length of 'TEXT': 2708
Maxn length of 'LABEL': 60
Maxn length of 'LABEL': 4


In [ ]:
'''# Load the tokenizer
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")'''

# Calculate max token length for the `abr` column
max_abr_token_length = dff['abr'].apply(lambda x: len(tokenizer.tokenize(x))).max()
print(f"Maximum token length in `abr`: {max_abr_token_length}")
# Calculate max token length for the `text` column
max_text_token_length = dff['text'].apply(lambda x: len(tokenizer.tokenize(x))).max()
print(f"Maximum token length in `text`: {max_text_token_length}")
# Calculate max token length for the `long` column
max_long_token_length = dff['long'].apply(lambda x: len(tokenizer.tokenize(x))).max()
print(f"Maximum token length in `long`: {max_long_token_length}")

Maximum token length in `abr`: 3
Maximum token length in `text`: 899
Maximum token length in `long`: 20


In [ ]:
'''dff = df1.sample(n=15, random_state=42)  # `random_state` ensures reproducibility'''

dff=df1
print(dff.shape)

(37500, 3)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Load model and move to device
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
# Tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
def split_into_chunks(text, tokenizer, chunk_size=512):
    # Ensure the input is a string, not a list
    if isinstance(text, list):
        text = " ".join(text)  # Join the list elements into a single string

    tokens = tokenizer.tokenize(text)  # Tokenize the text
    return [" ".join(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]

class AbbreviationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, chunk_size=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.chunk_size = chunk_size

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        input_text = f"{row['abr']} - {row['text']}"
        target_text = row["long"]

        # Split the input text into chunks
        chunks = split_into_chunks(input_text, self.tokenizer, chunk_size=self.chunk_size)

        # Tokenize input chunks and target text
        inputs = self.tokenizer(
            " ".join(chunks),  # Join the chunks back into a string
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0).to(device),
            "attention_mask": inputs["attention_mask"].squeeze(0).to(device),
            "labels": targets["input_ids"].squeeze(0).to(device),
        }

# Split DataFrame into train, validation, and test sets
train_df = dff.sample(frac=0.75, random_state=42)
v_df = dff.drop(train_df.index)
test_df = v_df.sample(frac=0.75, random_state=42)
valid_df = v_df.drop(test_df.index)
print(train_df.shape)
print(test_df.shape)
print(valid_df.shape)
# Create datasets
train_dataset = AbbreviationDataset(train_df, tokenizer)
valid_dataset = AbbreviationDataset(valid_df,tokenizer)
test_dataset = AbbreviationDataset(test_df, tokenizer)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

(28125, 3)
(7031, 3)
(2344, 3)


In [ ]:
"""#ok
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")"""
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs =

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
    model.save_pretrained("/content/drive/MyDrive/Abreviation/bart")
    tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart")

    # Validation phase
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(valid_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/4, Training Loss: 0.0027


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/4, Validation Loss: 0.0013
Validation Metrics: Accuracy: 0.7982, Precision: 0.7908, Recall: 0.7982, F1: 0.7713
1
Epoch 2/4, Training Loss: 0.0012


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 2/4, Validation Loss: 0.0007
Validation Metrics: Accuracy: 0.8468, Precision: 0.8636, Recall: 0.8468, F1: 0.8361
2
Epoch 3/4, Training Loss: 0.0009


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 3/4, Validation Loss: 0.0006
Validation Metrics: Accuracy: 0.8784, Precision: 0.9080, Recall: 0.8784, F1: 0.8701
3
Epoch 4/4, Training Loss: 0.0005


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, Validation Loss: 0.0006
Validation Metrics: Accuracy: 0.8793, Precision: 0.9098, Recall: 0.8793, F1: 0.8765


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Abreviation/bart")
tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart")



In [ ]:
# test phase
"""test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)"""
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, test Loss: 0.0006
Test Metrics: Accuracy: 0.8793, Precision: 0.9098, Recall: 0.8793, F1: 0.8765


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, train Loss: 0.0003
train Metrics: Accuracy: 0.9022, Precision: 0.9366, Recall: 0.9022, F1: 0.9042


In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart1")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart1")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# test phase
test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


Epoch 4/4, test Loss: 0.0036
Test Metrics: Accuracy: 0.6271, Precision: 0.5356, Recall: 0.6271, F1: 0.5449


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, train Loss: 0.0036
train Metrics: Accuracy: 0.6185, Precision: 0.5502, Recall: 0.6185, F1: 0.5355


In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart2")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart2")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# test phase
"""test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)"""
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, test Loss: 0.0011
Test Metrics: Accuracy: 0.7987, Precision: 0.8039, Recall: 0.7987, F1: 0.7728


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 4/4, train Loss: 0.0010
train Metrics: Accuracy: 0.8004, Precision: 0.8292, Recall: 0.8004, F1: 0.7764


In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart3")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart3")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# test phase
"""test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)"""
model.eval()
test_loss = 0
test_predictions = []
test_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


NameError: name 'epochs' is not defined

In [ ]:
def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1
epochs=5
print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


Epoch 5/5, test Loss: 0.0007
Test Metrics: Accuracy: 0.8500, Precision: 0.8797, Recall: 0.8500, F1: 0.8426


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, train Loss: 0.0005
train Metrics: Accuracy: 0.8615, Precision: 0.8890, Recall: 0.8615, F1: 0.8554


In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart4")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart4")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# test phase
"""test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)"""
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, test Loss: 0.0006
Test Metrics: Accuracy: 0.8810, Precision: 0.8959, Recall: 0.8810, F1: 0.8765


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, train Loss: 0.0003
train Metrics: Accuracy: 0.8921, Precision: 0.9223, Recall: 0.8921, F1: 0.8868


In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# test phase
"""test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)"""
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 5/5, test Loss: 0.0005
Test Metrics: Accuracy: 0.8899, Precision: 0.9196, Recall: 0.8899, F1: 0.8921


In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Abreviation/bart5")
# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/Abreviation/bart5")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs =1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
    model.save_pretrained("/content/drive/MyDrive/Abreviation/bart6")
    tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart6")

    # Validation phase
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(valid_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0005


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0006
Validation Metrics: Accuracy: 0.8993, Precision: 0.9118, Recall: 0.8993, F1: 0.8923


In [ ]:
# test phase
test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, test Loss: 0.0006
Test Metrics: Accuracy: 0.9016, Precision: 0.9144, Recall: 0.9016, F1: 0.8965


In [ ]:
# train results  bart51
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0002
train Metrics: Accuracy: 0.9218, Precision: 0.9411, Recall: 0.9218, F1: 0.9200


In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs =1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
    model.save_pretrained("/content/drive/MyDrive/Abreviation/bart6")
    tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart6")

    # Validation phase
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(valid_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0004


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0006
Validation Metrics: Accuracy: 0.9074, Precision: 0.9300, Recall: 0.9074, F1: 0.9077


In [ ]:
# test phase
test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, test Loss: 0.0005
Test Metrics: Accuracy: 0.9157, Precision: 0.9325, Recall: 0.9157, F1: 0.9168


In [ ]:
# train phase
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0002
train Metrics: Accuracy: 0.9347, Precision: 0.9553, Recall: 0.9347, F1: 0.9375


7 epoch

In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training and validation loop
epochs =1

def compute_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    return accuracy, precision, recall, f1
max_new_tokens=25
for epoch in range(epochs):
    # Training phase
    print(epoch)
    model.train()
    train_loss = 0
    j=1
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #print(j,"  loss: ",loss)
        loss.backward()
        optimizer.step()
        j=j+1
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")
    model.save_pretrained("/content/drive/MyDrive/Abreviation/bart51")
    tokenizer.save_pretrained("/content/drive/MyDrive/Abreviation/bart51")

    # Validation phase
    model.eval()
    val_loss = 0
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            # Decode predictions and labels for metric calculation
            predictions = model.generate(input_ids, attention_mask=attention_mask)
            val_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
            val_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss / len(valid_loader):.4f}")
    # Compute validation metrics
    metrics = compute_metrics(val_predictions, val_labels)
    print(f"Validation Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


0
Epoch 1/1, Training Loss: 0.0003


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, Validation Loss: 0.0005
Validation Metrics: Accuracy: 0.9177, Precision: 0.9366, Recall: 0.9177, F1: 0.9161


In [ ]:
# test phase
test_dataset = AbbreviationDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)
model.eval()
test_loss = 0
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        test_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        test_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, test Loss: {test_loss / len(test_loader):.4f}")
# Compute test metrics
metrics = compute_metrics(test_predictions, test_labels)
print(f"Test Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, test Loss: 0.0005
Test Metrics: Accuracy: 0.9243, Precision: 0.9393, Recall: 0.9243, F1: 0.9250


8 epoch

In [ ]:
# train results  bart51
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9467, Precision: 0.9632, Recall: 0.9467, F1: 0.9484


In [ ]:
#########################"micro
# train results  bart51
model.eval()
train_loss = 0
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        train_loss += outputs.loss.item()

        # Decode predictions and labels for metric calculation
        predictions = model.generate(input_ids, attention_mask=attention_mask)
        train_predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions])
        train_labels.extend([tokenizer.decode(label, skip_special_tokens=True) for label in labels])

print(f"Epoch {epoch + 1}/{epochs}, train Loss: {train_loss / len(train_loader):.4f}")
# Compute train metrics
metrics = compute_metrics(train_predictions, train_labels)
print(f"train Metrics: Accuracy: {metrics[0]:.4f}, Precision: {metrics[1]:.4f}, Recall: {metrics[2]:.4f}, F1: {metrics[3]:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch 1/1, train Loss: 0.0001
train Metrics: Accuracy: 0.9467, Precision: 0.9467, Recall: 0.9467, F1: 0.9467


In [ ]:
End